In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm
from sklearn.metrics import log_loss, accuracy_score
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.linear_model import LogisticRegression
import os
import shutil

Using TensorFlow backend.


In [43]:
#Since there are too many images and image classes, we'll consider only the top 16 classes
INPUT_SIZE = 300
NUM_CLASSES = 16
SEED = 1987
data_dir = os.getcwd()
labels = pd.read_csv(join(data_dir, 'labels.csv'))
print(len(listdir(join(data_dir, 'train'))), len(labels))
print(len(listdir(join(data_dir, 'test'))))

10222 10222
10357


In [44]:
selected_breed_list = list(labels.groupby('breed').count().sort_values(by='id', ascending=False).head(NUM_CLASSES).index)
labels = labels[labels['breed'].isin(selected_breed_list)]
labels['target'] = 1
labels['rank'] = labels['breed'].rank(ascending=0,method='dense')
labels_pivot = labels.pivot('id', 'breed', 'target').reset_index().fillna(0)
np.random.seed(seed=SEED)
rnd = np.random.random(len(labels))
train_idx = rnd < 0.8
valid_idx = rnd >= 0.8
y_train_val = labels_pivot[selected_breed_list].values
y_train = y_train_val[train_idx]
y_val = y_train_val[valid_idx]

In [45]:
#Reads and resizes the image, train_or_test is a string: "train"/"test"
def read_img(img_id, train_or_test, size):
    img = image.load_img(join(data_dir, train_or_test, '%s.jpg' % img_id), target_size=size)
    img = image.img_to_array(img)
    img = img/255.0
    return img

In [46]:
x_train_val = np.zeros((len(labels), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
for i, img_id in tqdm(enumerate(labels['id'])):
    img = read_img(img_id, 'train', (INPUT_SIZE, INPUT_SIZE))
    x = preprocess_input(np.expand_dims(img.copy(), axis=0))
    x_train_val[i] = x
print('Train Images shape: {} size: {:,}'.format(x_train_val.shape, x_train_val.size))

1777it [00:08, 205.94it/s]

Train Images shape: (1777, 300, 300, 3) size: 479,790,000


In [47]:
from tensorflow.keras.applications import VGG16
x_train = x_train_val[train_idx]
x_val = x_train_val[valid_idx]
print((x_train.shape, x_val.shape, y_train.shape, y_val.shape))
vgg_bottleneck = VGG16(weights='imagenet', include_top=False, pooling='avg')
train_vgg_bf = vgg_bottleneck.predict(x_train, batch_size=32, verbose=1)
valid_vgg_bf = vgg_bottleneck.predict(x_val, batch_size=32, verbose=1)
print('VGG train bottleneck features shape: {} size: {:,}'.format(train_vgg_bf.shape, train_vgg_bf.size))
print('VGG valid bottleneck features shape: {} size: {:,}'.format(valid_vgg_bf.shape, valid_vgg_bf.size))

((1409, 300, 300, 3), (368, 300, 300, 3), (1409, 16), (368, 16))
368/368 [==============================] - 6s 15ms/sample
VGG train bottleneck features shape: (1409, 512) size: 721,408
VGG valid bottleneck features shape: (368, 512) size: 188,416


In [50]:
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=SEED,max_iter = 9000)
logreg.fit(train_vgg_bf, (y_train * range(NUM_CLASSES)).sum(axis=1))
valid_probs = logreg.predict_proba(valid_vgg_bf)
valid_preds = logreg.predict(valid_vgg_bf)

In [51]:
print('Validation VGG LogLoss {}'.format(log_loss(y_val, valid_probs)))
print('Validation VGG Accuracy {}'.format(accuracy_score((y_val*range(NUM_CLASSES)).sum(axis=1), valid_preds)))

Validation VGG LogLoss 2.647898018386328
Validation VGG Accuracy 0.13858695652173914


In [52]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
x_train = x_train_val[train_idx]
x_val = x_train_val[valid_idx]
print((x_train.shape, x_val.shape, y_train.shape, y_val.shape))
inception_bottleneck = InceptionV3(weights='imagenet', include_top=False, pooling='avg')
train_i_bf = inception_bottleneck.predict(x_train, batch_size=32, verbose=1)
valid_i_bf = inception_bottleneck.predict(x_val, batch_size=32, verbose=1)
print('InceptionV3 train bottleneck features shape: {} size: {:,}'.format(train_i_bf.shape, train_i_bf.size))
print('InceptionV3 valid bottleneck features shape: {} size: {:,}'.format(valid_i_bf.shape, valid_i_bf.size))

((1409, 300, 300, 3), (368, 300, 300, 3), (1409, 16), (368, 16))
368/368 [==============================] - 2s 6ms/sample
InceptionV3 train bottleneck features shape: (1409, 2048) size: 2,885,632
InceptionV3 valid bottleneck features shape: (368, 2048) size: 753,664


In [54]:
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=SEED,max_iter = 9000)
logreg.fit(train_i_bf, (y_train * range(NUM_CLASSES)).sum(axis=1))
valid_probs = logreg.predict_proba(valid_i_bf)
valid_preds = logreg.predict(valid_i_bf)

In [55]:
print('Validation Inception LogLoss {}'.format(log_loss(y_val, valid_probs)))
print('Validation Inception Accuracy {}'.format(accuracy_score((y_val * range(NUM_CLASSES)).sum(axis=1), valid_preds)))

Validation Inception LogLoss 2.2721441854592306
Validation Inception Accuracy 0.30434782608695654
